# 1. Different editions of the game

## 1.1

In [6]:
import json

A = {
    'size': 100,
    'diesides' : 6,
    'snakes' : [(16,6), (47,26), (49,11 ), (56,53), (62,19), (64,60), (87,24), (93,73), (95,75), (98, 78)],
    'ladders' : [(1, 38), (4, 14), (9, 31), (28, 84), (21, 42), (36, 44), (51, 67), (71, 81), (80, 100)],
    'name' : "Milton Bradley",
    'URL' : "https://upload.wikimedia.org/wikipedia/en/b/ba/Cnl03.jpg"
}

B = {
    'size': 100,
    'diesides' : 6,
    'snakes' : [(73, 1), (46, 5), (55, 7), (48, 9), (59, 17), (83, 19), (44, 22), (95, 24), (98, 28), (69, 33), (64, 36), (92, 51)], 
    'ladders' : [(8, 26), (21, 82), (43, 77), (50, 91), (54, 93), (66, 87), (62, 96), (60, 100)],
    'name' : 'Random online example',
    'URL' : "http://2.bp.blogspot.com/-BAXe2vGFAEo/UxFsY8PwCAI/AAAAAAAAE0c/uHRwHjY8lwk/s1600/Snakes&Ladders-Nationalseriesgames.jpg"
}

C = {
    'size': 100,
    'diesides' : 6,
    'snakes' : [(6, 4), (10, 7), (20, 18), (38, 13), (44, 8), (52, 28), (68, 65), (74, 51), (82, 45), (98, 75)],
    'ladders' : [(2, 25), (19, 53), (21, 48), (32, 46), (43, 50), (59, 83), (66, 71), (79, 90), (89, 96), (94, 95)],
    'name' : 'custom',
    'URL' : None
}

with open('setups.json', 'a') as setups:
    json.dump(A, setups)
    json.dump(B, setups)
    json.dump(C, setups)

## 1.2

In [9]:
def validate_setup(d):
    """
    Validate keys and top-level types of values for game setup dictionary, raising a
      KeyError or TypeError if invalid, otherwise silent.
    Does not distinguish list or tuple types of snakes/ladders pairs.
    Does not check other data constraints, such as snakes or ladder pair values being
      not equal or clashing with another pair's values.
    """
    keys = sorted(d.keys())
    need_keys = sorted(['size', 'snakes', 'ladders', 'diesides', 'name', 'URL'])
    if keys != need_keys:
        raise KeyError("Correct keys not present in setup dictionary")
    if not isinstance(d['size'], int):
        raise TypeError("'size' must be int")
    if not isinstance(d['diesides'], int):
        raise TypeError("'diesides' must be int")
    active = [False]*101 # my code! used to make sure there is only one link per tile
    for kind in ('snakes', 'ladders'):
        if isinstance(d[kind], (list, tuple)):
            try:
                for i1, i2 in d[kind]:
                    if active[i1] or active[i2]: #raises an error when there's more than one link per square
                        raise TypeError("Maximum of one link per square")
                    active[i1]=True # mine. records that this square has a link
                    active[i2]=True # mine. same as above
                    # this loop is agnostic about type of sequence
                    if not isinstance(i1+i2, int):
                        raise TypeError("'{}' must be a sequence of pairs of ints".format(kind))
            except ValueError:
                # too many or not enough values to unpack from d['snakes']
                raise TypeError("'{}' must be a sequence of pairs of ints".format(kind))
        else:
            raise TypeError("'{}' must be a sequence of pairs of ints".format(kind))
    return "Valid!"

    
        
def standardize_setup(d):
    # don't standardize if valid
    validate_setup(d)
    r = d.copy() # copy
    r['snakes'] = [tuple(p) for p in d['snakes']]
    r['ladders'] = [tuple(p) for p in d['ladders']]
    return r

# You did this a different way in class, but I didn't want to bother to look it up and this gives the same result
# this actually compares any sort of nested lists, dicts, or tuples
def dict_compare(a, b):
    result = True
    for i in a:
        try: 
            if sorted(a[i]) == sorted(b[i]):
                pass
            else:
                result = False
        except TypeError:
            if a[i] == b[i]:
                pass
            else:
                result = False
    return result 

In [10]:
print(validate_setup(A))
print(validate_setup(B))
print(validate_setup(C))

print(dict_compare(A, B))
print(dict_compare(B, C))
print(dict_compare(A, C))

Valid!
Valid!
Valid!
False
False
False


# 2. Creating the Simulation Data

### 2.1

In [11]:
import pandas

snakes_ladders_df = pandas.DataFrame({'snakes' : [len(A['snakes']),len(B['snakes']),len(C['snakes'])], 
                                      'ladders' : [len(A['ladders']),len(B['ladders']),len(C['ladders'])]},
                                    index = ['A', 'B', 'C'])

print(snakes_ladders_df)

   ladders  snakes
A        9      10
B        8      12
C       10      10


### 2.2

In [18]:
import SnakesLadders as sl
from make_game import make_game_from_dict
import csv_utils

game_A = make_game_from_dict(A)
game_B = make_game_from_dict(B)
game_C = make_game_from_dict(C)

games=[game_A, game_B, game_C]
global i
i = 0
for game in games:
    labels = ['A', 'B', 'C']
    csv_utils.save_csv(csv_utils.multi_run(game, 1000), '{}_data.csv'.format(labels[i]))
    i += 1